In [2]:
import pandas as pd
import altair as alt
import numpy as np

## Analisis de Previo - contexto

* Los archivos que voy a extraer se encuentran alojados en gsheets en formato .xlsx, para eso creo una funcion que me ayude a extraerlos facilmente en el mismo formato

In [3]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

### Ceses y Rotacion
* Los *ceses* son la cantidad de colaboradores que dejan de trabajar en la empresa en un determinado mes
* La *rotacion* es el % entre los colaboradores cesados y el total de colaboradores
* Saber cuales son los ceses y su comportamiento ayuda a tener una vision general, pero esto no es comparable entre areas, zonas o cargos.
    * Por qué? Pues un cargo puede tener 10 ceses pero como tiene en total 100 colaboradores su rotacion solo le impacta un 10%; sin embargo si un cargo tiene 5 ceses pero su total de colaboradores es de 10, su rotacion sera 50%  es decir mas impactante
* Debido a esto es importante analizar el comportamiento tomando en cuenta los Ceses y la rotacion que generan

In [4]:
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
    cesados,
    usecols='E,I,O,P,S,T,W,X',
    dtype = {'CODIGO':str}
).rename(
    columns = {
        'Total o Temprana':'TIEMPO_CESE',
        'Producto':'PRODUCTO',
        'TIPO':'TIPO_CESE',
        'MOTIVO PRIMARIO':'MOTIVO_CESE'
    }
)

* Debido a un hecho extraordinario como una mudanza de una provincia a la capital los ceses del 2017 son atipicos, por lo se consideraran a partir del 2018
* Ademas el resto de bases parten desde este anio
* Es necesario transformar a inicio de mes la fecha de cese e ingreso porque es el intervalo que usare para comparar las temporalidades de otras bases

In [5]:
cesados.describe()

C:\Users\Anna\AppData\Local\Temp/ipykernel_11380/2700900083.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cesados.describe()


,CODIGO,CARGO,INGRESO,CESE,TIEMPO_CESE,PRODUCTO,TIPO_CESE,MOTIVO_CESE
count,3466,3466,3466,3466,3466,3466,3466,3466
unique,3446,11,537,988,2,2,2,25
top,A86AAAAAA3,VENTA GRUPAL,2018-06-04 00:00:00,2020-01-02 00:00:00,-,GRUPAL,VOLUNTARIA,Mejor Oferta De Trabajo
freq,2,1984,61,38,2837,2245,2338,583
first,NaN,NaN,1999-10-11 00:00:00,2017-01-02 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2021-11-19 00:00:00,2021-12-31 00:00:00,NaN,NaN,NaN,NaN


In [6]:
# Tranformando y filtrando lo necesario para el analisis
ceses = cesados.query('CESE >= "2018-01-01"')
ceses['CESE'] = ceses['CESE'].to_numpy().astype('datetime64[M]')
ceses['INGRESO'] = ceses['INGRESO'].to_numpy().astype('datetime64[M]')

C:\Users\Anna\AppData\Local\Temp/ipykernel_11380/3325222274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceses['CESE'] = ceses['CESE'].to_numpy().astype('datetime64[M]')
C:\Users\Anna\AppData\Local\Temp/ipykernel_11380/3325222274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceses['INGRESO'] = ceses['INGRESO'].to_numpy().astype('datetime64[M]')


#### Ceses
* Analizo los ceses separando el producto (Individual y Grupal) por ser diferente modelos de negocio dentro de la empresa
    * A primera vista, los ceses en Grupal son mas alta que en individual

In [7]:
ceses_prod = ceses.groupby(
    by=['PRODUCTO','CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count'))
# plot por producto
barra = alt.Chart(
    ceses_prod
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'year(CESE):N',
        title = None),
    y = alt.Y(
        'sum(cesados):Q',
        title = None,
        axis = None)
).properties(
    width = 300,
    height = 200
)
texto = barra.mark_text(
    align = 'center',
    baseline ='top',
    dy = 5,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('sum(cesados)')
)
# plot barra  y texto
(barra+texto).facet('PRODUCTO').properties(
    title = 'Total de Ceses por Producto Acumulado por Año'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
)


alt.FacetChart(...)

* Analizo el comportamineto de los ceses segun los cargos en cada tipo de producto
    * Se ve calaramente que la fuerza de ventas (a diferecnia de los otros cargos) es la que genera mayor cantidad de ceses
    * Esto podria deberse a que tambien son los cargos con mayor cantidad de personas activas, para esto sera necesaria analizar la rotacion por cargo

In [8]:
ceses_cargo = ceses.groupby(
    by=['PRODUCTO', 'CARGO', 'CESE'],
    as_index = False
    ).agg(
        cesados = ('CODIGO', 'count')
    )

alt.Chart(
    ceses_cargo
).mark_bar().encode(
    x = alt.X(
        'year(CESE):N',
        title = None),
    y = alt.Y(
        'sum(cesados):Q',
        title = None),
    color = alt.Color('CARGO:O'),
    column = alt.Column(
        'PRODUCTO',
        title = None)
).properties(
    width = 300,
    height = 200,
    title = 'Total ceses por Cargo y Producto Acumulado por Año'
).configure_view(
    strokeWidth = 0 #quita el borde
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
)

alt.Chart(...)

#### Rotacion
* Para analizar la rotacion necesitare unir mi base de planilla con la de ceses para sacar el % de ceses entre el total de colaboradores
* Esto me ayudara a saber si la hipotesis de que los cargos de Venta Grupal e Individual son los mas importantes de analizar por tener la mayor cantidad de colaboradores cesados y una alta rotacion que impacta a la empresa

In [9]:
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
    planilla,
    usecols='A,B,D:H',
    dtype={'COD':str}
).rename(
    columns={
        'COD':'CODIGO',
        'REGION/DPTO':'REGION',
        'DIVISION/GC':'DIVISION'

    }
)

* Uno la base de ceses con la de planilla agrupada hasta nivel cargo para visualizar la rotacion por mes
* Para este caso lo que quiero comprar es la rotacion anual (mensualmente seria un periodo muy corto y probablemente muy volatil), la cual esta compuesta por la suma de las rotaciones mensuales, para ello creo la columna year

In [10]:
head_cargo = planilla.groupby(
    by=['PRODUCTO', 'CARGO', 'MES'],
    as_index = False
    ).agg(
        real = ('CODIGO', 'count')
    )
## Join ceses y planilla
rot_cargo = pd.merge(
    head_cargo,
    ceses_cargo,
    left_on=['MES', 'PRODUCTO', 'CARGO'],
    right_on=['CESE', 'PRODUCTO', 'CARGO'])
# hallo la rotacion cesados/real
rot_cargo['rotacion'] = np.where(
    rot_cargo['real'] == 0,
    0,
    ((rot_cargo['cesados'] / rot_cargo['real'])*100).round(1))
    
rot_cargo['year'] = rot_cargo['MES'].to_numpy().astype('datetime64[Y]')

* Grafico la rotacion de los cargos del Producto Grupal e Individual agrupado de manera anual
    * Se muestra que los cargos de Ventas Individual y Ventas Grupal tuvieron una alta rotacion constante en estos ultimos años
    * Hasta el momento concluimos que el analisis se debe hacer, en una primera etapa, a los cargos de ventas, ya que se tendria un impacto mayor en la rotacion de la compañia.
    * Asi mismo, estos dos cargos son la fuerza de ventas de la empresa, la cual tiene que mantenerse con una buena covertura

In [11]:
#### Grafico Grupal
rot_cargo_g = rot_cargo.query('PRODUCTO == "GRUPAL"'
    ).groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index = False
    ).agg(
        rotacion = ('rotacion','sum')
    )
## Grafico
barra_g = alt.Chart(
    rot_cargo_g
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.Y(
        'year(year):N',
        title = None),
    y = alt.X(
        'rotacion:Q',
        title = None,
        axis = None)
).properties(
    width = 200,
    height = 200
)

texto_g = barra_g.mark_text(
    align = 'center',
    baseline ='top',
    dy = 3,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('rotacion')
)

gru = (barra_g + texto_g).facet(
    'CARGO',
    columns = 6
).properties(
    title = '%Rotacion Anual por Cargo de Grupal')

#### Grafico Individual
rot_cargo_i = rot_cargo.query('PRODUCTO == "INDIVIDUAL"'
    ).groupby(
        by = ['year', 'PRODUCTO', 'CARGO'],
        as_index = False
    ).agg(
        rotacion = ('rotacion','sum')
    )
barra_i = alt.Chart(
    rot_cargo_i
).mark_bar(
    color = '#04328C'
).encode(
    x = alt.Y(
        'year(year):N',
        title = None),
    y = alt.X(
        'rotacion:Q',
        title = None,
        axis = None)
).properties(
    width = 200,
    height = 200
)

texto_i = barra_i.mark_text(
    align = 'center',
    baseline ='top',
    dy = 3,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('rotacion')
)

ind = (barra_i + texto_i).facet(
    'CARGO',
    columns = 5
).properties(
    title = '%Rotacion Anual por Cargo de Individual')

##### Uniendo ambos graficos
alt.vconcat(gru, ind).configure_view(
    stroke = '#04B2D9',
    strokeWidth = 0.3 
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#04328C'
)

alt.VConcatChart(...)

### Antiguedad de los Cesados - Ventas Individual y Grupal
* En la empresa considera como ceses tempranos a los que cesan por cualquier motivo antes de los 6 meses
* Seria importante ver el comportamiento en antiguedad de los ceses para saber en que mes empiezan cesar mas, si se vuelve constante en algun momento, en general, que comportamiento tienen la cantidad de ceses con la antiguedad de los ceses
* Para esto, necesitamos hallar la antiguedad en cantidad de meses, la definimos como fecha de CESE - fecha INGRESO

In [12]:
ceses = ceses.query('CARGO == "VENTA INDIVIDUAL" | CARGO == "VENTA GRUPAL"')
ceses['antiguedad_meses'] = (((ceses['CESE'] - ceses['INGRESO']).dt.days)/30).round(0)

* Grafico el total de ceses segun la antiguedad en meses de ambos cargos separados por tipo de cese (Voluntario o Involuntario), para vista general
    * Se ve una diferencia en el comportamiento en la antiguedad de ceses voluntarios vs ceses involuntarios
    * Los colores mas claros reflejan la clasificacion que la empresa le da a los ceses menores de 6 meses como Tempranos
    * En los ceses **involuntarios** el comportamiento durante los 9 primeros meses es diferente a los ceses de meses mayores o iguales a 12, donde ya se puede ver un comportamiento que disminuye en el tiempo
    * En los ceses **voluntarios** el comportamiento diferente se da solo los en 3 primeros meses, del mes 6 adelante los ceses disminuyen con el tiempo
    * Debido a este comportamiento diferente se debe hacer una analisis y prediccion para cada tipo de cese. **Considerando que los ceses involuntarios podrian deberse a despidos por diferente motivos, por lo que el analisis repercutiria en otro proyecto que vaya mas alineado al tema de seleccion, ya que los colaboradores contratados no cumplen con la expectativa de la empresa.**
    * Por lo tanto, para este proyecto se tomaran solo los ceses voluntarios. **Para este caso, al parecer la clasificacion de antiguedad que toma la empresa (6 meses) seria incorrecta, pues el cambio de comportamiento se da desde el mes 3**. Seria necesario un analisi por cargo de Ventas Grupal e Individual para validar esta hipotesis

In [13]:
ceses_antg = ceses.groupby(
    by=['TIPO_CESE','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count'))

# GRAFICANDO
alt.Chart(ceses_antg).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses'),
    color = alt.condition(
        alt.datum.antiguedad_meses < 6.0, 
        alt.value('#04B2D9'),
        alt.value('#04328C')),
    column = alt.Column(
        'TIPO_CESE',
        title = None)
).properties(
    title = 'Total de Ceses Segun Antiguedad por Producto',
    width = 500,
    height = 250
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.Chart(...)

* Analizando solo el comportamiento de Ceses Voluntarios por Ventas Grupal e Individual
    * Para los Vendedores de **Grupal** se muestra un comportamiento de a mayor antiguedad menor cantidad de ceses, esto a partir del mes 3
    * Sin embargo, para los Vendedores **Individuales** en el comportamiento es alparecer bimodal. **Desde el mes 3 la cantidad de ceses se reduce con el tiempo, pero desde el mes 12 empieza a subir nuevamente hasta el mes 21, donde recien empieza a tener un comportamiento que s reduce con el tiempo**
    * Hasta el momento el analisis en ventas **Grupal** se tomaran en cuenta los ceses a partir del mes 3
    * En ventas **Individual** por ahora se tomarian 2 grupos despues de los 3 meses.
        * El primer grupo del rango de igual a 3 meses hasta 12 meses
        * El segundo grupo de igual a 12 meses hasta 21 meses 
        * El ultimo grupo de igual a 21 meses a mas

In [14]:
ceses = ceses.query('TIPO_CESE == "VOLUNTARIA"')

In [15]:
ceses_antg_g = ceses.groupby(
    by=['PRODUCTO','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
).query('PRODUCTO == "GRUPAL"'
)

# GRAFICANDO grupal
barra_g = alt.Chart(ceses_antg_g).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
)
linea_g_3 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [3]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

gru = (barra_g + linea_g_3
).properties(title = 'Total de Ceses Voluntarios Segun Antiguedad - Grupal',
    width = 600,
    height = 250
)
############
ceses_antg_i = ceses.groupby(
    by=['PRODUCTO','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
).query('PRODUCTO == "INDIVIDUAL"'
)

# GRAFICANDO indiviudal
barra_i = alt.Chart(ceses_antg_i).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
)
linea_i_3 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [3]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

linea_i_12 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [12]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

linea_i_21 = alt.Chart(
    pd.DataFrame({'antiguedad_meses': [21]})
).mark_rule(
    color = 'red',
    strokeWidth = 2
).encode(
    x = 'antiguedad_meses')

ind = (barra_i + linea_i_3 + linea_i_12 + linea_i_21).properties(
    title = 'Total de Ceses Voluntarios Segun Antiguedad - Individual',
    width = 600,
    height = 250
)
### juntando graficos
alt.vconcat(gru,ind
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.VConcatChart(...)

### Motivos de Cese Voluntarios - Ventas Grupal e Individual
* La empresa cuenta con una clasificacion de motivos de cese que se genra despues de la entrevista de salida
* Este analisis se realizara de manera seraparada para ventas Grupal e Individual
* Estos motivos de cese podrian ayudarme a segmentar mas el comportamiento del grupo que quiero analizar

#### Ventas Grupal
* Tomo como medida de compracion a la mediana de la antiguedad, ya que la distribucion de los meses de antiguedad en ambos casos es mas dispersa y no tienen un comportamiento normal

In [16]:
ceses_g = ceses.query('PRODUCTO == "GRUPAL" and antiguedad_meses >= 3')

* En el caso de Grupal se muestran los motivos con menor antiguedad en Personal Eventual y No le gusto el trabajo
    * Los **motivos de Personal Eventual, No le gusto el trabajo y falsas expectativas** las consideraremos como posibles errores en la seleccion de personal por una mala explicacion en las funciones del cargo, es por ellos que suceden dentro de los primeros meses de antiguedad
    * Los otros motivos depende de la empresa, estos pueden ser mejorados con capacitacion, salario acorde a mercado o brindando facilidades y beneficios a los colaboradores
    * Sin embargo, la mediana del motivo de Falsas Expectativas es mas alta por lo que tiene mas relacion con los otros motivos, queda la duda si deberia ser utilizado o no en el analisis.
        *  EL histograma de falsas expectativas muestra un comportamiento mas volatil a traves de los meses
        *  Analizando un poco este motivo podria deberse a colaboradores que tienen una expectativa en un rango de tiempo mas amplio que el resto (crecimiento profesional, beneficios de antiguedad, etc), por lo que luego de mas de 2 años cesan por este motivo
* Por lo concluido anteriormente para este analisis no se tomaran en cuenta los motivos de ceses de Personal Eventual y No Le Gusto el Trabajo


In [17]:
motivo_cese = ceses_g.groupby(
    by = ['MOTIVO_CESE'],
    as_index = False
).agg(
    antg_prom = ('antiguedad_meses', 'median')
)

barra = alt.Chart(motivo_cese).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort='x',
        title = 'Motivo de Ceses Voluntarios',),
    x = alt.X(
        'antg_prom',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.MOTIVO_CESE == 'Falsas Expectativas' , 
        alt.value("#04B2D9"),
        alt.value("#04328C"))
)

texto = barra.mark_text(
    align = 'center',
    baseline ='bottom',
    dy = 8,
    dx = 12,
    color = '#F2F2F2',
    fontSize = 12,
    fontWeight = 'bold'
).encode(
    text = alt.Text('antg_prom',format =',.2r')
)

motivos = (barra+texto).properties(
    title = 'Mediana de Antiguedad de Ceses por Motivos Voluntarios',
    width = 400,
    height = 300
)
### falsas
motivo_cese_falexp = ceses_g.groupby(
    by=['MOTIVO_CESE','antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
).query('MOTIVO_CESE == "Falsas Expectativas"'
)

barra = alt.Chart(motivo_cese_falexp).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'sum(cesados)',
        title = None,
        axis = None)
).properties(
    title = 'Ceses por Falsas Expectativas por Antiguedad',
    width = 400,
    height = 250
)

texto = barra.mark_text(
    align = 'center',
    baseline ='top',
    dy = 3,
    color = '#F2F2F2',
    fontSize = 13,
    fontWeight = 'bold'
).encode(
    text = alt.Text('sum(cesados)')
)

falsas = (barra + texto)

## uniendo
alt.hconcat(motivos, falsas).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.HConcatChart(...)

#### Ventas Individual

In [18]:
ceses_i = ceses.query('PRODUCTO == "INDIVIDUAL" and antiguedad_meses >= 3')

* En el caso de individual los **motivos de Personal Eventual, No le gusto el trabajo y falsas expectativas** se presentan con menor antiguedad media, por lo que podriamos concluir que la mayoria de ceses suceden dentro de los 12 primeros meses
* En este caso es mas logico separar estos tres motivos del resto, debido a su concepto y a su antiguedad.
* Debido a lo anterior, para el caso de Ventas Individual no se tomaran en cuenta los motivos de Personal Eventual, No le Gusto el trabajo y Falsas expectativas


In [19]:
motivo_cese = ceses_i.groupby(
    by = ['MOTIVO_CESE'],
    as_index = False
).agg(
    antg_median = ('antiguedad_meses', 'median')
)

barra = alt.Chart(motivo_cese).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort='x',
        title = 'Motivo de Ceses Voluntarios',),
    x = alt.X(
        'antg_median',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.antg_median <= 12, 
        alt.value("#04B2D9"),
        alt.value("#04328C"))
).properties(
    title = 'Mediana Antiguedad por Motivo de Cese - Individual',
    width = 500,
    height = 390
)

texto = barra.mark_text(
    align = 'center',
    baseline ='bottom',
    dy = 8,
    dx = 12,
    color = '#F2F2F2',
    fontSize = 12,
    fontWeight = 'bold'
).encode(
    text = alt.Text('antg_median',format =',.2r')
)

motivos = (barra+texto).properties(
    title = 'Mediana de Antiguedad de Ceses por Motivos Voluntarios',
    width = 400,
    height = 300
)

(barra+texto).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.LayerChart(...)

#### Nueva Distribucion de antiguedad
* Con los cambios aplicados en  Grupal e Individual, lo mas probable es que la distribucion de antiguedad haya cambiado (sobretodo en Individual)
* Esto podria llevar a un cambio de agrupacion

In [20]:
motivos_grupal = ['Personal Eventual', 'No le Gusto el Trabajo']
ceses_g = ceses_g.query('MOTIVO_CESE not in @motivos_grupal')
motivos_individual = ['Personal Eventual', 'No le Gusto el Trabajo', 'Falsas Expectativas']
ceses_i = ceses_i.query('MOTIVO_CESE not in @motivos_individual')

* La nueva distribucion de Grupal se va dividir en dos grupos posibles para el analisis
    * Un grupo de 3 a 9 meses, este grupo por tener un rango de meses muy pequeño, puedo considerarlo como parte de la adaptacion a la cultura de la empresa, no seria prioritario para este analisis
    * El segundo grupo generado es de 9 a mas meses, donde ya se puede ven un comportamiento que reduce los ceses en el tiempo
* La distribucion de Individual se va dividir en dos grupos
    * Un grupo de 3 a 21 meses donde el cese disminuye con la antiguedad
    * Un grupo de 21 a mas meses donde la antiguedad los ceses tienen otro pico y vuelve a disminuir en el tiempo

In [21]:
ceses_antg_g = ceses_g.groupby(
    by=['antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
)

# GRAFICANDO grupal
barra_g = alt.Chart(ceses_antg_g).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
).properties(title = 'Total de Ceses Voluntarios Segun Antiguedad - Grupal',
    width = 600,
    height = 250
)
############
ceses_antg_i = ceses_i.groupby(
    by=['antiguedad_meses'],
    as_index=False
).agg(
    cesados = ('CODIGO', 'count')
)

# GRAFICANDO indiviudal
barra_i = alt.Chart(ceses_antg_i).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'cesados',
        title = 'Total Ceses')
).properties(
    title = 'Total de Ceses Voluntarios Segun Antiguedad - Individual',
    width = 600,
    height = 250
)
### juntando graficos
alt.vconcat(barra_g,barra_i
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.VConcatChart(...)

### Conclusiones
* De todo el universo de colaboradores de producto Individual y Grupal, solo se analizaran los Vendedores Grupal e Individual por ser los de mayor impacto en cada producto
* Solo se considerara a los colaboradores cesados con ciertas antiguedades
    * Grupal: Cesados con mas de 9 meses de antiguedad
    * Individual: Un grupo con mas de 21 meses de antiguedad y otro grupo de 3 a 21 meses
* Se consideraran solo algunos motivos en cada caso:
    * Grupal: Todos los motivos excepto Personal Eventual y  No le gusto el trabajo
    * Individual: Todos los motivos excepto Personal Eventual, No le gusto el trabajo y falsas expectativa